In [1]:
import os
import pickle
from ibm_watson_studio_pipelines import WSPipelines

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
from ibm_botocore.client import Config
import ibm_boto3


def download_file_cos(local_file_name,key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=API_KEY_MLOPS,
                           ibm_service_instance_id=CRN_MLOPS,
                           ibm_auth_endpoint=AUTH_ENDPOINT,
                           config=Config(signature_version='oauth'),
                           endpoint_url=ENDPOINT_URL_MLOPS)
    try:
        res=cos.download_file(Bucket=BUCKET_MLOPS,Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
        
def load_loss(key, filename):
    download_file_cos(key,filename)
    with open (filename,"rb") as f:
        loss = pickle.load(f)
    return loss

In [5]:
train_loss = load_loss('train_loss.pkl','train_loss.pkl')

File Downloaded


In [6]:
train_loss = train_loss['binary_logloss']

## Test if the model training converges

In [7]:
def check_loss_converge(loss):
    # check every 5 iterations if the loss has decreased
    coverges = [loss[i]>loss[i+4] for i in range(0, len(loss),5)]
    return all(coverges)

In [12]:
model_converges = check_loss_converge(train_loss)

In [13]:
model_converges

True

## Save the test results to pipeline

In [14]:
test_params = {'train_converge': model_converges}

In [15]:
pipelines_client = WSPipelines.from_apikey(apikey=apikey)
pipelines_client.store_results(test_params)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "train_converge": .ibm_watson_studio_pipelines/results/train_converge
